<a href="https://colab.research.google.com/github/jan-de-trop/Billboard-Top-100-Singles/blob/main/Data_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing the Libraries

In [1]:
#Import libraries
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import json
import re
import requests

## Billboard Top 100 Singles Scraping and Parsing

In [2]:
#Import Beautiful Soup
from bs4 import BeautifulSoup

We're scraping [Wikipedia](http://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_) to extract the Billboard top 100 singles and their rankings. 

To do this, create a list of dictionaries, 100 of them to be precise, with the following entries: {'url': '/wiki/Sugarloaf_(band)', 'ranking': 30, 'band_singer': 'Sugarloaf', 'title': 'Green-Eyed Lady'}.

Here, url is the link for the singer or band, ranking is the ranking of the song, band_singer is the name of the artist(s) and title is the title of the song.

The text from the requests are stored in a dictionary. This dictionary as its keys the years (as integers from 1992 to 2020), and as values corresponding to these keys the text of the page being fetched.

In [3]:
years=range(1992, 2021)
yearstext={}
for y in years:
    yreq=requests.get("http://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_%i" % y)
    yearstext[y]=yreq.text
    time.sleep(1)

### Parsing the data

In [4]:
"""
Function
--------
parse_year

Inputs
------
year: the year we want the singles for
ytextdixt: a dictionary with keys as integer years and values the downloaded web pages 
    from wikipedia for that year.
   
Returns
-------

a list of dictionaries, each of which corresponds to a single and has the
following data:

Eg:

{'band_singer': ['Brandy', 'Monica'],
  'ranking': 2,
  'song': ['The Boy Is Mine'],
  'songurl': ['/wiki/The_Boy_Is_Mine_(song)'],
  'titletext': '" The Boy Is Mine "',
  'url': ['/wiki/Brandy_Norwood', '/wiki/Monica_(entertainer)']}
  
A dictionary with the following data:
    band_singer: a list of bands/singers who made this single
    song: a list of the titles of songs on this single
    songurl: a list of the same size as song which has urls for the songs on the single 
        (see point 3 above)
    ranking: ranking of the single
    titletext: the contents of the table cell
    band_singer: a list of bands or singers on this single
    url: a list of wikipedia singer/band urls on this single: only put in the part 
        of the url from /wiki onwards
    
"""

fields = ["ranking", "song", "songurl", "titletext", "band_singer", "url"]

def get_cols(row):
    return row.find_all("th") + row.find_all("td") 

def break_a(col):
    #Finds the a tag
    return list(map(list, zip(*[(a.get("title").strip('"'), a.get("href")) for a in col.find_all("a")]) )) or [[col.get_text().strip('"')], [None]]
    
def parse_cols(cols):
    #Ranking, song & songurl, titletext(with "), band singer & band URL
    return [int(cols[0].get_text())] + break_a(cols[1]) + [cols[1].get_text()] + break_a(cols[2])

def create_dict(cols):
    return dict(zip(fields, cols))

# Parser function.
def parse_year(year, yearstext):
    soup = BeautifulSoup(yearstext[year], 'html.parser')
    rows = soup.find("table", attrs={"class": "wikitable"}).find_all("tr")[1:]
    return [create_dict(parse_cols(get_cols(row))) for row in rows]

In [5]:
yearinfo = {y: parse_year(y, yearstext) for y in years}

We'll store this in a json file so we don't have to run it every time.

In [9]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [10]:
fd = open("/content/gdrive/MyDrive/yearinfo.json","w")
json.dump(yearinfo, fd)
fd.close()
del yearinfo

In [11]:
#Loading the dictionary back to the same variable
with open("/content/gdrive/MyDrive/yearinfo.json", "r") as fd:
    yearinfo = json.load(fd)

## Constructing a Dataframe

In [12]:
yeardict={}
for y in yearinfo.keys():
    yearlist=yearinfo[y]
    yearlist2=[]
    for idict in yearlist:
        singers=idict['band_singer']
        for i,s in enumerate(singers):
            songs=idict['song']
            for j,so in enumerate(songs):#now inside each singer song combination
                nd={}
                nd['band_singer']=s
                nd['url']=idict['url'][i]
                nd['song']=so
                nd['songurl']=idict['songurl'][j]
                nd['ranking']=idict['ranking']
                yearlist2.append(nd)
    yeardict[y]=pd.DataFrame(yearlist2)#one for each year

yearspanel=pd.concat(yeardict, axis=0).reset_index()
yearspanel=yearspanel.sort_values(by=['ranking','level_0'])
flatframe=yearspanel.reset_index()
del flatframe['level_1']
del flatframe['index']
flatframe = flatframe.rename(columns={'level_0':'year'})
flatframe.head(10)

,year,band_singer,url,song,songurl,ranking
0,1992,Boyz II Men,/wiki/Boyz_II_Men,End of the Road (Boyz II Men song),/wiki/End_of_the_Road_(Boyz_II_Men_song),1
1,1993,Whitney Houston,/wiki/Whitney_Houston,I Will Always Love You,/wiki/I_Will_Always_Love_You#Whitney_Houston_v...,1
2,1994,Ace of Base,/wiki/Ace_of_Base,The Sign (song),/wiki/The_Sign_(song),1
3,1995,Coolio,/wiki/Coolio,Gangsta's Paradise,/wiki/Gangsta%27s_Paradise,1
4,1995,L.V. (singer),/wiki/L.V._(singer),Gangsta's Paradise,/wiki/Gangsta%27s_Paradise,1
5,1996,Los del Río,/wiki/Los_del_R%C3%ADo,Macarena (song),/wiki/Macarena_(song),1
6,1997,Elton John,/wiki/Elton_John,Something About the Way You Look Tonight,/wiki/Something_About_the_Way_You_Look_Tonight,1
7,1997,Elton John,/wiki/Elton_John,Candle in the Wind 1997,/wiki/Candle_in_the_Wind_1997,1
8,1998,Next (group),/wiki/Next_(group),Too Close (Next song),/wiki/Too_Close_(Next_song),1
9,1999,Cher,/wiki/Cher,Believe (Cher song),/wiki/Believe_(Cher_song),1


In [13]:
flatframe.dtypes

year           object
band_singer    object
url            object
song           object
songurl        object
ranking         int64
dtype: object

In [14]:
flatframe['year']=flatframe['year'].astype("int")
flatframe['band_singer']=flatframe['band_singer'].astype("string")
flatframe['url']=flatframe['url'].astype("string")
flatframe['song']=flatframe['song'].astype("string")
flatframe['songurl']=flatframe['songurl'].astype("string")
flatframe.dtypes

year            int64
band_singer    string
url            string
song           string
songurl        string
ranking         int64
dtype: object

Converting the dataframe to a pickle file

In [15]:
flatframe.to_pickle('/content/gdrive/MyDrive/flatframedf')
del flatframe

In [16]:
#Loading in the dataframe
flatframe = pd.read_pickle('/content/gdrive/MyDrive/flatframedf')
flatframe.head()

,year,band_singer,url,song,songurl,ranking
0,1992,Boyz II Men,/wiki/Boyz_II_Men,End of the Road (Boyz II Men song),/wiki/End_of_the_Road_(Boyz_II_Men_song),1
1,1993,Whitney Houston,/wiki/Whitney_Houston,I Will Always Love You,/wiki/I_Will_Always_Love_You#Whitney_Houston_v...,1
2,1994,Ace of Base,/wiki/Ace_of_Base,The Sign (song),/wiki/The_Sign_(song),1
3,1995,Coolio,/wiki/Coolio,Gangsta's Paradise,/wiki/Gangsta%27s_Paradise,1
4,1995,L.V. (singer),/wiki/L.V._(singer),Gangsta's Paradise,/wiki/Gangsta%27s_Paradise,1


## Scraping and Parsing Wikipedia for artists' info

A cache object `urlcache` that will avoid redundant HTTP requests (e.g. an artist might have multiple singles on a single year, or be on the list over a span of years) is created. This function is designed to be used in a loop over years, and then a loop over songs per year. Since network requests are relatively slow, if we have already requested for a singer or band's wikipedia page, caching the results speeds up scraping for hundreds of artist webpages.

We have wrapped the call in _an exception block_. If the request gets an HTTP code different from 200, the cells for that URL will have a value of 1; and if the request completely fails (e.g. no network connection) the cell will have a value of 2. This will allow you to analyse the failed requests.

In [17]:
urlcache={}
def get_page(url):
    if (url not in urlcache) or (urlcache[url]==1) or (urlcache[url]==2):
        time.sleep(1)
        try:
            r = requests.get("http://en.wikipedia.org%s" % url)
            if r.status_code == 200:
                urlcache[url] = r.text
            else:
                urlcache[url] = 1
        except:
            urlcache[url] = 2
    return urlcache[url]

In [18]:
flatframe=flatframe.sort_values('year')
flatframe.head()

,year,band_singer,url,song,songurl,ranking
0,1992,Boyz II Men,/wiki/Boyz_II_Men,End of the Road (Boyz II Men song),/wiki/End_of_the_Road_(Boyz_II_Men_song),1
1822,1992,KWS (band),/wiki/KWS_(band),Please Don't Go (KC and the Sunshine Band song),/wiki/Please_Don%27t_Go_(KC_and_the_Sunshine_B...,47
2096,1992,Michael Bolton,/wiki/Michael_Bolton,When a Man Loves a Woman (song),/wiki/When_a_Man_Loves_a_Woman_(song),54
633,1992,Guns N' Roses,/wiki/Guns_N%27_Roses,November Rain,/wiki/November_Rain,17
2555,1992,Michael Jackson,/wiki/Michael_Jackson,In the Closet,/wiki/In_the_Closet,66


In [19]:
flatframe["url"].apply(get_page)

0       <!DOCTYPE html>\n<html class="client-nojs" lan...
1822    <!DOCTYPE html>\n<html class="client-nojs" lan...
2096    <!DOCTYPE html>\n<html class="client-nojs" lan...
633     <!DOCTYPE html>\n<html class="client-nojs" lan...
2555    <!DOCTYPE html>\n<html class="client-nojs" lan...
                              ...                        
758     <!DOCTYPE html>\n<html class="client-nojs" lan...
759     <!DOCTYPE html>\n<html class="client-nojs" lan...
3097    <!DOCTYPE html>\n<html class="client-nojs" lan...
3179    <!DOCTYPE html>\n<html class="client-nojs" lan...
3909    <!DOCTYPE html>\n<html class="client-nojs" lan...
Name: url, Length: 3910, dtype: object

In [20]:
#Let us make sure that there are no unresolved pages
#The sum below should be 0, and the boolean True. If that is not the case, run the above cell again until you get a sum of 0 and a boolean True
print (np.sum([(urlcache[k]==1) or (urlcache[k]==2) and isinstance(k,str) for k in urlcache]))
print (len(flatframe.url.unique())==len(urlcache))#we got all of the urls

0
True


In [22]:
#Let's save the `urlcache` and remove the old object. 
keys_values = urlcache.items()
urlcache = {str(key): str(value) for key, value in keys_values}
with open("/content/gdrive/MyDrive/artistinfo.json","w") as fd:
    json.dump(urlcache, fd)
del urlcache

In [23]:
with open("/content/gdrive/MyDrive/artistinfo.json") as json_file:
    urlcache = json.load(json_file)

## Parsing the Data

Things kept in mind: 

Wikipedia has defined the same genre in a few different ways. Our parsing code will pick these up as different and new as they all differ with the alphabet case or an underscore instead of a hyphen.

We have defined a function to ensure that we have the correct representation of the number of artists in a particular genre.

In [24]:
genres_duplicates={'/wiki/Adult_Contemporary_music':'/wiki/Adult_contemporary','/wiki/Adult_contemporary_music':'/wiki/Adult_contemporary',
'/wiki/Afrobeat':'/wiki/Afrobeats',
'/wiki/Alternative_rock':'/wiki/Alternative_Rock',
'/wiki/Avant-garde':'/wiki/Avant-garde_music',
'/wiki/Blues':'/wiki/Blues_music',
'/wiki/Comedy_hip-hop':'/wiki/Comedy_hip_hop',
'/wiki/Contemporary_R%26B':'/wiki/Contemporary_R%26B_music',
'/wiki/Contemporary_folk':'/wiki/Contemporary_folk_music',
'/wiki/Country_Folk':'/wiki/Country_folk',
'/wiki/Dance_pop':'/wiki/Dance-pop',
'/wiki/East_Coast_hip_hop':'/wiki/East_coast_hip_hop',
'/wiki/Electronic_Dance_Music':'/wiki/Electronic_dance_music',
'/wiki/Electronica':'/wiki/Electronica_music',
'/wiki/Emo':'/wiki/Emo_music',
'/wiki/Electropop':'/wiki/Electro-pop',
'/wiki/Folk-pop':'/wiki/Folk_pop',
'/wiki/Funk':'/wiki/Funk_music',
'/wiki/Grime_(music_genre)':'/wiki/Grime_music',
'/wiki/Gangsta_Rap':'/wiki/Gangsta_rap',
'/wiki/Hip_Hop_music': '/wiki/Hip_hop','/wiki/Hip_hop_music':'/wiki/Hip_hop',
'/wiki/Hyphy':'/wiki/Hyphy_music',
'/wiki/Latin_music':'/wiki/Latin_music_(genre)',
'/wiki/West_Coast_hip_hop':'/wiki/West_coast_hip_hop',
'/wiki/Southern_Hip_Hop':'/wiki/Southern_hip_hop',
'/wiki/Ska':'/wiki/Ska_music',
'/wiki/Pop-rock':'/wiki/Pop_rock',
'/wiki/Pop_Music':'/wiki/Pop_music',
'/wiki/Nu_metal':'/wiki/Nu_metal_music',
'/wiki/Hard_Rock':'/wiki/Hard_rock',
'/wiki/Pop_Rock':'/wiki/Pop_rock',
'/wiki/Post-Grunge':'/wiki/Post-grunge',
'/wiki/SoundCloud_rap':'/wiki/Soundcloud_rap'}

def genre_duplicates(genres):
    for i in range(len(genres)):
        if genres[i] in genres_duplicates.keys():
            genres[i]=genres_duplicates[genres[i]]
    return genres 

In [25]:
# Function to get the longest active years for an artist
def most_active_years(ya):
    max_years=0
    for i,k in enumerate(ya):
        x=k.split('–')
        if 'present' in x[1].lower():
            x[1]=2021
        if max_years<int(x[1])-int(x[0]):
            index=i
            max_years = int(x[1])-int(x[0])
    return index

In [33]:
"""
Function
--------
singer_band_info

Inputs
------
url: the url
page_text: the text associated with the url
   
Returns
-------
A dictionary with the following data:
    url: copy the input argument url into this value
    genres: the genres that the band or singer works in
    born: the artist's birthday
    ya: years active variable
        we'll use regex to parse the years

"""
def singer_band_info(url, page_text):
    genres=['NA']
    born=None
    ya=None
    try:
        soup = BeautifulSoup(page_text, "html.parser")
        rows=soup.find("table",attrs={"class": "infobox"}).find_all("tr")
        for i,r in enumerate(rows):
            if 'Genre' in r.text:
                genres=[]
                g=rows[i].find("td")
                gen=[i.get("href") for i in g.find_all("a") if 'cite_note' not in i.get("href")]
                genres=genre_duplicates(gen)
            if 'Born' in r.text and rows[i].find('span',attrs={"class": "bday"})!=None:
                born=rows[i].find('span',attrs={"class": "bday"}).text.strip(' ').strip('(').strip(')')
            if 'Years' in r.text:
                ya=rows[i].find("td")
                text=[text for text in ya.find_all(text=True) if text.parent.name not in ['a','span']]
                ya=''.join(text)
                ya = re.findall("[0-9]{4}[–][0-9]{4}", ya) + re.findall("[0-9]{4}[–][a-zA-Z]{7}", ya)
                if ya !=[]:
                  ind=most_active_years(ya)
                  ya=ya[ind]
        print(dict(url=url,genres=genres, born=born, ya=ya))
        return(dict(url=url,genres=genres, born=born, ya=ya))
    except (AttributeError,TypeError): 
      pass

In [34]:
singer_band_info_list=[]
for k,v in urlcache.items():
  x=singer_band_info(k, v)
  if x!=None:
      singer_band_info_list.append(x)

In [31]:
#Store the list and delete the variable
with open("/content/gdrive/MyDrive/singer_band_info.json","w") as fd:
    json.dump(singer_band_info_list, fd)
del singer_band_info_list

In [32]:
#Load the list into a variable
with open("/content/gdrive/MyDrive/singer_band_info.json") as json_file:
    singer_band_info_list = json.load(json_file)

In [35]:
tempdf=pd.DataFrame(singer_band_info_list)
tempdf.head()

,url,genres,born,ya
0,/wiki/Boyz_II_Men,"[/wiki/Contemporary_R%26B_music, /wiki/New_jac...",None,1987–present
1,/wiki/KWS_(band),"[/wiki/Pop_music, /wiki/Electronica_music, /wi...",None,1991–1994
2,/wiki/Michael_Bolton,"[/wiki/Pop_rock, /wiki/Soft_rock, /wiki/Blue-e...",1953-02-26,1975–present
3,/wiki/Guns_N%27_Roses,"[/wiki/Hard_rock, /wiki/Heavy_metal_music]",None,1985–present
4,/wiki/Michael_Jackson,"[/wiki/Pop_music, /wiki/Soul_music, /wiki/Funk...",1958-08-29,1964–2009


Note that the 
`
born
`
column shows None for the artists that perform as bands.


##Creating a Dataframe 

In [36]:
largedf=flatframe.merge(tempdf, on="url")
largedf.head()

,year,band_singer,url,song,songurl,ranking,genres,born,ya
0,1992,Boyz II Men,/wiki/Boyz_II_Men,End of the Road (Boyz II Men song),/wiki/End_of_the_Road_(Boyz_II_Men_song),1,"[/wiki/Contemporary_R%26B_music, /wiki/New_jac...",None,1987–present
1,1992,Boyz II Men,/wiki/Boyz_II_Men,It's So Hard to Say Goodbye to Yesterday,/wiki/It%27s_So_Hard_to_Say_Goodbye_to_Yesterday,37,"[/wiki/Contemporary_R%26B_music, /wiki/New_jac...",None,1987–present
2,1992,Boyz II Men,/wiki/Boyz_II_Men,Uhh Ahh,/wiki/Uhh_Ahh,84,"[/wiki/Contemporary_R%26B_music, /wiki/New_jac...",None,1987–present
3,1993,Boyz II Men,/wiki/Boyz_II_Men,In the Still of the Night (1956 song),/wiki/In_the_Still_of_the_Night_(1956_song)#Bo...,12,"[/wiki/Contemporary_R%26B_music, /wiki/New_jac...",None,1987–present
4,1994,Boyz II Men,/wiki/Boyz_II_Men,I'll Make Love to You,/wiki/I%27ll_Make_Love_to_You,3,"[/wiki/Contemporary_R%26B_music, /wiki/New_jac...",None,1987–present


In [37]:
genres = set()
for g in largedf.genres:
    genres.update(g)
genres = sorted(genres)

#Make a column for each genre
for genre in genres:
      largedf[genre] = [(genre in singergenres)*1 for singergenres in largedf.genres]         

In [38]:
largedf['ya']=largedf['ya'].astype("string")
largedf['url']=largedf['url'].astype("string")
largedf['genres']=largedf['genres'].astype("string")
largedf['born']=largedf['born'].astype("string")

In [39]:
largedf.to_pickle('/content/gdrive/MyDrive/largedf')
del largedf

In [40]:
largedf = pd.read_pickle('/content/gdrive/MyDrive/largedf')
largedf.head()

,year,band_singer,url,song,songurl,ranking,genres,born,ya,#Musical_style,/w/index.php?title=Soul_rock_(music)&action=edit&redlink=1,/wiki/Acid_jazz,/wiki/Acid_rock,/wiki/Acoustic_music,/wiki/Acoustic_rock,/wiki/Adult_contemporary,/wiki/Afrobeats,/wiki/Alternative_R%26B,/wiki/Alternative_Rock,/wiki/Alternative_country,/wiki/Alternative_dance,/wiki/Alternative_folk,/wiki/Alternative_hip_hop,/wiki/Alternative_metal,/wiki/Alternative_music,/wiki/Alternative_pop,/wiki/Ambient_music,/wiki/Americana_(music),/wiki/Anarcho-punk,/wiki/Anti-folk,/wiki/Arena_rock,/wiki/Art_pop,/wiki/Art_rock,/wiki/Atlantic_Records,/wiki/Avant-funk,/wiki/Avant-garde_music,/wiki/Avant-pop,/wiki/Bachata_(music),/wiki/Baroque_pop,/wiki/Bass_music,...,/wiki/Space_rock,/wiki/Speed_metal,/wiki/Spoken_word,/wiki/Surf_music,/wiki/Swing_music,/wiki/Synth-pop,/wiki/Synth-rock,/wiki/Synthpop,/wiki/Synthwave_(2000s_genre),/wiki/Tech_house,/wiki/Techno,/wiki/Teen_pop,/wiki/Thrash_metal,/wiki/Traditional_pop_music,/wiki/Trance_music,/wiki/Trap_music,/wiki/Trap_music_(EDM),/wiki/Trap_music_(hip_hop),/wiki/Trip_hop,/wiki/Trop_rock,/wiki/Tropical_house,/wiki/Tropical_music,/wiki/UK_bass,/wiki/UK_funky,/wiki/UK_garage,/wiki/Underground_hip_hop,/wiki/Urban_adult_contemporary,/wiki/Urban_contemporary,/wiki/Urban_contemporary_gospel,/wiki/Urban_pop,/wiki/Urbano_music,/wiki/Video_game,/wiki/Wagnerian_rock,/wiki/West_coast_hip_hop,/wiki/Western_swing,/wiki/Working_on_Dying#Tread_style,/wiki/World_music,/wiki/Worldbeat,/wiki/Worship_music,NA
0,1992,Boyz II Men,/wiki/Boyz_II_Men,End of the Road (Boyz II Men song),/wiki/End_of_the_Road_(Boyz_II_Men_song),1,"['/wiki/Contemporary_R%26B_music', '/wiki/New_...",<NA>,1987–present,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1992,Boyz II Men,/wiki/Boyz_II_Men,It's So Hard to Say Goodbye to Yesterday,/wiki/It%27s_So_Hard_to_Say_Goodbye_to_Yesterday,37,"['/wiki/Contemporary_R%26B_music', '/wiki/New_...",<NA>,1987–present,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1992,Boyz II Men,/wiki/Boyz_II_Men,Uhh Ahh,/wiki/Uhh_Ahh,84,"['/wiki/Contemporary_R%26B_music', '/wiki/New_...",<NA>,1987–present,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1993,Boyz II Men,/wiki/Boyz_II_Men,In the Still of the Night (1956 song),/wiki/In_the_Still_of_the_Night_(1956_song)#Bo...,12,"['/wiki/Contemporary_R%26B_music', '/wiki/New_...",<NA>,1987–present,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1994,Boyz II Men,/wiki/Boyz_II_Men,I'll Make Love to You,/wiki/I%27ll_Make_Love_to_You,3,"['/wiki/Contemporary_R%26B_music', '/wiki/New_...",<NA>,1987–present,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
